<a href="https://colab.research.google.com/github/EkaterinaVoloshina/NNMethods/blob/main/HW4_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
!pip install transformers

In [69]:
import transformers
from transformers import BertModel, AutoTokenizer, BertTokenizer, PreTrainedTokenizerFast, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F

import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from torch import nn, optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader

Используем BERT-tiny, чтобы тратить меньше времени на обучение модели:

In [70]:
PRE_TRAINED_MODEL_NAME = 'prajjwal1/bert-tiny'

# Загрузка и обработка данных

Этот раздел содержит код по подготовке датасета из семинара

In [71]:
!gdown --id 1S6qMioqPJjyBLpLVz4gmRTnJHnjitnuV
!gdown --id 1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv

Downloading...
From: https://drive.google.com/uc?id=1S6qMioqPJjyBLpLVz4gmRTnJHnjitnuV
To: /content/apps.csv
100% 134k/134k [00:00<00:00, 69.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv
To: /content/reviews.csv
100% 7.17M/7.17M [00:00<00:00, 100MB/s]


In [72]:
df = pd.read_csv("reviews.csv")

In [73]:
def to_sentiment(rating):
  rating = int(rating)
  if rating <= 2:
    return 0
  elif rating == 3:
    return 1
  else: 
    return 2

df['sentiment'] = df.score.apply(to_sentiment)

In [74]:
class GPReviewDataset(Dataset):

  def __init__(self, reviews, labels, tokenizer, max_len):
    self.reviews = reviews
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.reviews)
  
  def __getitem__(self, item):
    review = str(self.reviews[item])
    label = self.labels[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',
      truncation=True
    )

    return {
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'label': torch.tensor(label, dtype=torch.long)
    }

In [75]:
def create_dataset(df, tokenizer, max_len):
  ds = GPReviewDataset(
    reviews=df.content.to_numpy(),
    labels=df.sentiment.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return ds

In [76]:
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [77]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

In [78]:
MAX_LEN = 160
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

train_dataset = create_dataset(df_train, tokenizer, MAX_LEN)
val_dataset = create_dataset(df_val, tokenizer, MAX_LEN)
test_dataset = create_dataset(df_test, tokenizer, MAX_LEN)

loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/585ac1c3dedc6b808dd35e8770afafe10905d3e723a02617af749d39db780e09.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": nul

# Базовая модель

Возьмем архитектуру с последнего семинара и добавим подсчет лосса внутрь класса, чтобы она работала с Trainer:

In [79]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes, loss_fn):
    super().__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.loss_fn = loss_fn
  
  def forward(self, input_ids, attention_mask, labels):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False)
    output = self.drop(pooled_output)
    output = self.out(output)
    loss = loss_fn(output, labels)
    return (loss, output)

In [80]:
loss_fn = nn.CrossEntropyLoss()
model = SentimentClassifier(n_classes=3, loss_fn=loss_fn)
model = model.to(device)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/1ee037c9e1a220d5c814779ffe697080d1e6f5b16

# Определим гиперпараметры и метрики

Зададим метрики и гиперпараметры, которые будем использовать в дальнейшем:

In [81]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [82]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


# Тренировка базовой модели

Теперь натренируем нашу базовую модель:

In [83]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)


In [84]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14171
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2215


Step,Training Loss
10,1.137100
20,1.126800
30,1.138000
40,1.141800
50,1.156600
60,1.134400
70,1.149000
80,1.129200
90,1.136100
100,1.138900


Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./results/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./results/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./results/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2215, training_loss=0.8509842061027716, metrics={'train_runtime': 226.9937, 'train_samples_per_second': 312.145, 'train_steps_per_second': 9.758, 'total_flos': 0.0, 'train_loss': 0.8509842061027716, 'epoch': 5.0})

Посмотрим теперь на качество на тестовой выборке. В целом модель справляется на 0.67 accuracy. Кажется, что у модели могут быть проблемы с каким-то одним классом.

In [85]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 787
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.6772554002541296,
 'eval_f1': 0.6753262512077274,
 'eval_loss': 0.7251932621002197,
 'eval_precision': 0.6749195116651341,
 'eval_recall': 0.6772554002541296,
 'eval_runtime': 1.4771,
 'eval_samples_per_second': 532.807,
 'eval_steps_per_second': 16.925}

# Модель, объединяющая CLS и выход пуллер-слоя

Теперь добавим CLS-токен с последнего слоя, прогоним через дропаут и возьмем среднее от пуллер-слоя и CLS-токена.

In [86]:
class CLSClassifier(nn.Module):

  def __init__(self, n_classes, loss_fn):
    super().__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.loss_fn = loss_fn
  
  def forward(self, input_ids, attention_mask, labels):
    last_hidden_state, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False)
    output = self.drop(pooled_output)
    cls = self.drop(last_hidden_state[:,0,:])
    output = torch.stack([output, cls]).mean(dim=0)
    output = self.out(output)
    loss = loss_fn(output, labels)
    return (loss, output)

In [87]:
combined_classifier = CLSClassifier(n_classes=3, loss_fn=loss_fn)
combined_classifier = combined_classifier.to(device)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/1ee037c9e1a220d5c814779ffe697080d1e6f5b16

Тренируем:

In [88]:
combined_trainer = Trainer(
    model=combined_classifier,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

combined_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14171
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2215


Step,Training Loss
10,1.143800
20,1.138900
30,1.142800
40,1.097000
50,1.160600
60,1.146200
70,1.133500
80,1.119900
90,1.143400
100,1.109800


Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./results/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./results/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./results/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2215, training_loss=0.8319077536968291, metrics={'train_runtime': 208.1555, 'train_samples_per_second': 340.395, 'train_steps_per_second': 10.641, 'total_flos': 0.0, 'train_loss': 0.8319077536968291, 'epoch': 5.0})

Посмотрим, как работает. Кажется, на 0.02 лучше, чем предыдущая, но все еще достаточно плохо.

In [89]:
combined_trainer.evaluate()

***** Running Evaluation *****
  Num examples = 787
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.6937738246505718,
 'eval_f1': 0.6892474747522559,
 'eval_loss': 0.7020341753959656,
 'eval_precision': 0.686502124115098,
 'eval_recall': 0.6937738246505718,
 'eval_runtime': 1.4499,
 'eval_samples_per_second': 542.786,
 'eval_steps_per_second': 17.242}

# Готовая модель из 🤗 

In [90]:
from transformers import BertForSequenceClassification

In [91]:
bert_model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=3)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggi

Тренируем:

In [92]:
bert = Trainer(
    model=bert_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

bert.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14171
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2215


Step,Training Loss
10,1.106000
20,1.104200
30,1.110100
40,1.108000
50,1.099000
60,1.108300
70,1.103400
80,1.093200
90,1.100700
100,1.091400


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2215, training_loss=0.8296175001975375, metrics={'train_runtime': 208.0123, 'train_samples_per_second': 340.629, 'train_steps_per_second': 10.648, 'total_flos': 28140136939200.0, 'train_loss': 0.8296175001975375, 'epoch': 5.0})

Посмотри на качество. Оно хуже, чем у предыдущей модели, но приблизительно такое же, как у первой модели

In [93]:
bert.evaluate()

***** Running Evaluation *****
  Num examples = 787
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.6785260482846251,
 'eval_f1': 0.6769092136826292,
 'eval_loss': 0.7108674049377441,
 'eval_precision': 0.676218437530164,
 'eval_recall': 0.6785260482846251,
 'eval_runtime': 1.4708,
 'eval_samples_per_second': 535.094,
 'eval_steps_per_second': 16.998}

# Модель с CLS токенами разных слоев

Теперь возьмем CLS-токены со всех слоев и возьмем среднее от них как аутпут:

In [94]:
class AllLayersClassifier(nn.Module):

  def __init__(self, n_classes, loss_fn):
    super().__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME, 
                                          output_hidden_states=True)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.loss_fn = loss_fn
  
  def forward(self, input_ids, attention_mask, labels):
    last_hidden_state, pooled_output, hidden_states = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False)
    cls = [state[:,0,:] for state in hidden_states]
    output = torch.stack(cls).mean(dim=0)
    output = self.out(output)
    loss = loss_fn(output, labels)
    return (loss, output)

In [95]:
all_layer_classifier = AllLayersClassifier(n_classes=3, loss_fn=loss_fn)
all_layer_classifier = all_layer_classifier.to(device)

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/1ee037c9e

Тренируем:

In [96]:
all_layer_trainer = Trainer(
    model=all_layer_classifier,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

all_layer_trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14171
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2215


Step,Training Loss
10,1.171100
20,1.141800
30,1.169800
40,1.124800
50,1.173500
60,1.141300
70,1.101000
80,1.109700
90,1.126300
100,1.108000


Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./results/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./results/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./results/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2215, training_loss=0.8168746063456309, metrics={'train_runtime': 207.5506, 'train_samples_per_second': 341.387, 'train_steps_per_second': 10.672, 'total_flos': 0.0, 'train_loss': 0.8168746063456309, 'epoch': 5.0})

Качество ближе к модели, которая интегрирует CLS-токен и пуллер-слой. Кажется, CLS-токены вносят небольшое улучшение качества. Так как это tiny модель, у нее только два слоя, и это не улучшило качество заметно.

In [97]:
all_layer_trainer.evaluate()

***** Running Evaluation *****
  Num examples = 787
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.6950444726810674,
 'eval_f1': 0.6941123256243968,
 'eval_loss': 0.7023467421531677,
 'eval_precision': 0.693384369587427,
 'eval_recall': 0.6950444726810674,
 'eval_runtime': 1.4657,
 'eval_samples_per_second': 536.932,
 'eval_steps_per_second': 17.056}

# Сравнение качества моделей на отзывах

Посмотрим на трех отзывах, как ведут себя модели:

In [98]:
negative = """The new update is terrible. It glitches, makes my phone die really 
easily (it's not a problem for my phone, when I go on my tablet, computer, iPad, 
and my other phones, it still has the same problem) and it takes so much space 
when it updates. this has caused me to uninstall the game. 
The app used to be really good before the update though."""

neutral = """Lovely, fun game but a few problems: The glitches where coins disappear, especially after
using the jetpack sometimes. Jumping in super sneakers by the entrance of a tunnel, 
trying to collect a powerup only to crash into the top of the tunnel and not 
even gaining the power up. Placing the weekly hunt objects on places unreachable, 
like the top of a tunnel without super sneakers (I get that it's a challenge 
but this one is too much). Fixing these glitches would make this game get 5 stars."""

positive = """this is a fun game to play when you are bored or have nothing to do. 
I like that you can verse other people on face book and they can set a goal 
for you to try and beat. I also like that you can unlock new characters and try 
to save up money or keys to get them. there are different kinds of each character 
qich means the same character but in different costumes. there are also different 
things (characters, scate bords etc.) that you can buy only in some kind of 
time but it costs lots. thanksalot"""

In [99]:
test_dataset = [negative, neutral, positive]

In [100]:
def get_results(trainer, dataset, tokenizer, max_len=160):
    test_dataset = GPReviewDataset(
    reviews=dataset,
    labels=[0, 1, 2],
    tokenizer=tokenizer,
    max_len=max_len
    )
    preds = trainer.predict(test_dataset=test_dataset)
    probs = torch.from_numpy(preds[0]).softmax(1)
    predictions = probs.numpy()
    results = list(np.argmax(predictions,axis=1))
    return results

In [101]:
first_model_results = get_results(trainer, test_dataset, tokenizer)
second_model_results = get_results(combined_trainer, test_dataset, tokenizer)
third_model_results = get_results(bert, test_dataset, tokenizer)
fourth_model_results = get_results(all_layer_trainer, test_dataset, tokenizer)

***** Running Prediction *****
  Num examples = 3
  Batch size = 32


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Prediction *****
  Num examples = 3
  Batch size = 32


***** Running Prediction *****
  Num examples = 3
  Batch size = 32


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Prediction *****
  Num examples = 3
  Batch size = 32


In [102]:
df = pd.DataFrame({
    'texts': test_dataset,
    'labels': [0, 1, 2],
    'first_model': first_model_results,
    'second_model': second_model_results,
    'third_model': third_model_results,
    'fourth_model': fourth_model_results
})

In [103]:
df

,texts,labels,first_model,second_model,third_model,fourth_model
0,"The new update is terrible. It glitches, makes...",0,0,0,0,0
1,"Lovely, fun game but a few problems: The glitc...",1,1,2,1,2
2,this is a fun game to play when you are bored ...,2,1,1,1,1


Как мы видим, ни одна модель